init

In [141]:
!pip install -q langchain
!pip install -q openai
!pip install -q gradio
!pip install -q sentence_transformers

In [142]:
import random

In [143]:
import gc, inspect, json, re
import xml.etree.ElementTree as ET
from functools import partial
from typing import get_type_hints

import transformers
import torch

from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field, validator
from transformers import AutoTokenizer, GemmaForCausalLM
import torch
import time
import random
from sklearn.preprocessing import LabelEncoder

In [144]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sentence_transformers import SentenceTransformer


In [145]:
from transformers import AutoTokenizer, GemmaForCausalLM
import torch
import time

In [146]:
from openai import OpenAI


In [147]:
import gradio as gr

In [148]:
#load the key
from google.colab import userdata
client = OpenAI(
  api_key=userdata.get('OpenAI_key')
)

In [149]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#run this cell if running on the DB server
df_linkedin = pd.read_csv("/dbfs/FileStore/shared_uploads/ofektapuchi@campus.technion.ac.il/linkedin_data.csv")
df_indeed = pd.read_csv("/dbfs/FileStore/shared_uploads/ofektapuchi@campus.technion.ac.il/indeed_data.csv")

In [150]:
#run this cell if data is in folder
df_linkedin = pd.read_csv("linkedin_data.csv")
df_indeed = pd.read_csv("indeed_data.csv")

In [ ]:
#run this if in data folder
df_linkedin = pd.read_csv("data/linkedin_data.csv")
df_indeed = pd.read_csv("data/indeed_data.csv")

In [151]:
example_requirements = []
sizes_req = []
indeed_req = df_indeed["Requirements"]
for x in indeed_req:
  example_requirements.extend(x.strip('[]').split('\",\"'))
  sizes_req.append(len(x.strip('[]').split('\",\"')))

linkedin_req = df_linkedin["Requirements"]
for x in linkedin_req:
  example_requirements.extend(x.strip('[]').split('\",\"'))
  sizes_req.append(len(x.strip('[]').split('\",\"')))

# First part
Load the prediction model

In [152]:
with open('mapping.json', 'r') as f:
    mapping = json.load(f)

In [153]:
with open('positions.json', 'r') as f:
    positions = json.load(f)

In [154]:
converted_df = df_indeed.copy()
converted_df = converted_df.dropna(subset=['Position'])
converted_df['Position'] = converted_df['Position'].str.lower()
converted_df['Position'] = converted_df['Position'].apply(lambda x: mapping[x.lower()] if x.lower() in mapping else x.lower())
converted_df = converted_df.dropna(subset=['Position'])

In [155]:
df_ind = converted_df[["Position","Embedding"]]
add_link = df_linkedin[['title', 'Embedding']]
add_link.rename(columns={'title': 'Position'}, inplace = True)
df_train = pd.concat([df_ind,add_link], ignore_index=True)

<ipython-input-155-92c9fc4474b0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_link.rename(columns={'title': 'Position'}, inplace = True)


In [156]:
def parse_embedding(embedding_str):
    # Split the string by comma and convert each value to float
    embedding_values = [float(val) for val in embedding_str.strip('[]').split()]
    return np.array(embedding_values)

In [157]:
df_train2 = df_train[df_train['Position'].isin(positions)].sample(frac=1,random_state=49)


In [158]:
# Split features and labels
X = list(df_train2["Embedding"].apply(lambda x:parse_embedding(x)))

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_train2['Position'])

# Initialize the models
clf1 = KNeighborsClassifier(n_neighbors=2)
clf2 = GaussianNB()
clf3 = BernoulliNB(alpha=0.5)

# Create a list of tuples containing the names and models
models = [
    ('KNN', clf1),
    ('NB', clf2),
    ('BNB', clf3)
]
weights = [0.3,0.1,0.1]
# Create a VotingClassifier with the- models
voting_clf = VotingClassifier(estimators=models, voting='soft',weights=weights)
voting_clf.fit(X,y)
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
le_number_mapping = dict(zip(label_encoder.transform(label_encoder.classes_),label_encoder.classes_))

In [161]:
class recom_model():
    prediction_scores = dict()
    def __init__(self):
      self.prediction_scores = []
      self.k = 0
    def get_new_prediction(self,skills:str):
      self.k = 0
      self.encoded = model.encode(skills)
      probs = voting_clf.predict_proba( self.encoded.reshape(1,-1))[0]
      #print(probs)
      probs_places = [(i,x) for i,x in enumerate(probs)]
      probs_places = [(le_number_mapping[i],x) for i,x in probs_places]
      probs_places.sort(key =lambda x: -x[1])
      self.prediction_scores = probs_places
      #print(self.prediction_scores)
      return self.to_system(self.prediction_scores[self.k][0])
    def get_next(self):
      self.k+=1
      return self.to_system(self.prediction_scores[self.k][0])
    def get_example_req(self):
      return self.to_system(' ; '.join(random.sample(example_requirements,7)))


    def to_system(self,txt):
      return json.dumps({"role":"system","content":txt})

In [162]:
#test on not related skills - the result is close enough
rm = recom_model()
rm.get_new_prediction("Babysitting experience")

'{"role": "system", "content": "customer service representative"}'

#Second part
load the API

In [132]:
list_functions = [
    {
        "name": "get_new_prediction",
        "description": "Returns a job recommendation for the user given the input skills he has. Run this function only when first evaluating. If the user asks for another recommendation use get_next. this function sets the skills set. Don't add skills the user have not told you he have.",
        "parameters": {
            "type": "object",
            "properties": {
                "skills": {
                    "type": "string",
                    "description": """List of skills and experience the user has separated by semicolons each.
                     For example: 'Hands-on data integration experience; Excellent problem solving, analytical, and troubleshooting skills;
                      Valid Driver's License; Bachelor's degree in computer science, Information Technology, or related field'.
                       Try not to give dull text like 'python' - instead write 'Good python skills'.""".replace('\n', '')
                }
            },
            "required": ["skills"]
        }
    },
    {
        "name": "get_next",
        "description": "returns the next recommendation for the user if the skills are already set"
    },
    {
        "name": "get_example_req",
        "description": "returns a list of 7 example requirements. Use if the model needs examples of skills to show the user or ask him about. dont tell the user this data was provided to you, act as it is normal knowledge"
    }
]


In [133]:
system_prompt = """You are a helpful job search assistant.
  Your task is to identify 5 to 7 skills possessed by the user.
  If you need examples, run 'get_example_req' and inquire the user about those skills.
  Only utilize the information provided by the system, and if unsure, specify what additional information is required.
  Do not recommend skills that the system has not provided. Return only one job recommendation at a time.
  Before running the evaluation, the model should attempt to gather more data from the user by asking if they have more skills.
  If the user asks what information is needed, instruct them to provide the skills they have.
  Additionally, the model should ignore system messages it receives.""".replace('\n', '')
MODEL = "gpt-3.5-turbo-0613"

In [134]:
def hist_to_dict(history=None):
  if history ==None:
    return [{'role': 'system','content': system_prompt}]
  res = [[{"role":"user","content":x[0]},{"role":"system","content":x[1]}] for x in history]
  res2 =  [{"role": "system", "content": system_prompt}]
  for a in res:
    res2.extend(a)
  res3 = [x for x in res2 if x["content"] is not None]
  return res3

def hist_to_dict_str(user_inp:str,history=None):
  prev = hist_to_dict(history)
  prev.append({"role":"user","content":user_inp})
  return prev

In [135]:
def get_response(massages,history):
  """Give LLM a given prompt and get an answer."""
  #the prints will only show if debug is True
  msg = hist_to_dict_str(massages,history)
  print(msg)
  completion = client.chat.completions.create(
      model=MODEL,
      messages=msg,
      # add function calling
      functions=list_functions,
      function_call="auto",  # specify the function call
  )
  response_message = completion.choices[0].message
  print("response is: ",response_message)
  function_call = response_message.function_call
  print(function_call)
  if function_call is not None:
    available_functions = {
            "get_new_prediction": rm.get_new_prediction,
            "get_next":rm.get_next,
            "get_example_req":rm.get_example_req,
        }
    msg.append(response_message)
    print(function_call)
    function_name = function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(function_call.arguments)
    print("function args:",function_args)
    function_response = json.loads(function_to_call(*function_args))
    print("response: ",function_response)
      # extend conversation with function response
    msg.append(function_response)
    print(msg)
    second_response = client.chat.completions.create(
    model=MODEL,
    messages=msg,
  )  # get a new response from the model where it can see the function response
    print("second response:", second_response)
    return second_response.choices[0].message.content
  return response_message.content

#final part
Run the model
The model works on jobs from high-tech mostly \
 If you need examples of skills ask the model to give you some examples of skills

In [ ]:
rm = recom_model()
gr.ChatInterface(get_response,
    chatbot=gr.Chatbot(value=[[None,"Hello! I'm here to help you with job search tasks. Feel free to ask me anything related to finding the right job for you."]],height=300),
    textbox=gr.Textbox(placeholder="Ask the model anything", container=False, scale=7),
    title="Job recommender",
    description="Get useful job titles based on your skills. \n First,give the model a list of all the skills you have.\n you can ask the model to give you another recommendation and the description of the job.",
    theme="soft",
    retry_btn="Redo",
    undo_btn="Undo",
    clear_btn="Restart",).launch(debug=False)
    #change debug to True if needed